# Víceruký bandita

viz [Bayesian Methods for Hackers](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers) str. 164

##### Zadání úlohy: 

* Vstupujete do kasína s kýblem $n_{coins}$ mincí. 
* V kasínu je $n_{bandits}$ výherních mašin. 
* Každá mašina vrací vhozenou minci s neznámou pravděpodobností $p_i$. 
* Z kasína si nesmíte odnést žádnou minci, se kterou vstupujete, ale můžete si odnést všechny mince, které vám mašiny vrátí. 
* Úkolem je odnést si co nejvíce.


In [ ]:
from bandits import Bandits

bandits = Bandits(n_bandits = 5)

print(bandits)

## Strategie

Strategie říká, jak vybrat banditu pro vhození další mince.

Strategie je implementována jako funktor inicializovaný parametrem:

* `n_bandits` — celkový počet banditů

Strategie implementuje metodu `__call__` s parametry:

* `thrown` — pole s počtem mincí dříve vhozených do mašin
* `returned` — pole s počtem mincí dříve vrácených mašinami


#### Připravené strategie

##### RegularStrategy

* Strategie pravidelně střídá mašiny bez ohledu na jejich výstup

##### RandomStrategy

* Strategie náhodně vybírá mašiny  bez ohledu na jejich výstup






In [ ]:
from bandits import Strategy
from bandits import BayesStrategy, RepeatStrategy

class RegularStrategy(Strategy):
    """ samples bandits regulary 1,2,3... """
    
    def __call__(self, thrown, returned):
        return sum(thrown) % n_bandits
    
    
class RandomStrategy(Strategy):
    """ samples bandits randomly """
    
    def __call__(self, thrown, returned):
        return np.random.randint(0, n_bandits, 1)[0]

#### Test strategie




In [ ]:
# Doplňte kód pro výpočet návratnosti pro každou strategii po vhození i mincí

import numpy as np
import pandas as pd
import copy


n_bandits = 5
n_coins = 1000


bandits =                                 #! vytvořte množinu banditů
print(bandits)

strategies = [RegularStrategy(n_bandits), RandomStrategy(n_bandits)]

returns =                                 #! inicializace matice coins x strategies

for j                                     #! pro každou strategii j  

    bandits.reset()                       #! reset vnitřního stavu banditů
    for i                                 #! pro každou minci
        bandits.throw()                   #! hoď minci do bandity podle strategie
        returns[i, j] =                   #! spočti aktuální návratnost

df_ret = pd.DataFrame(returns, columns=[s.name() for s in strategies])

print(df_ret.head())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

palette = ['#4285F4','#EA4335','#FBBC05','#34A853']

plt.figure(figsize=(12,4))
sns.lineplot(data=pd.melt(df_ret.reset_index(), var_name='strategy', id_vars='index'),
             x ='index', y='value', hue='strategy', palette=palette[:df_ret.shape[1]], 
             ).set(title='Návratový poměr')
plt.axhline(max([b.rate for b in bandits.bandits]), color='black', linestyle='--', label= 'optimal strategy')
plt.legend(loc='lower right');

## Experimentální porovnání

* Porovnání strategií v kontrolovaném experimentu

* Všem strategiím bude opakovaně předložena stejná sada náhodně generovaných bandidtů
* Výsledky budou porovnány napříč experimenty

#### Parametry experimentu

* `n_bandits` — počet banditů
* `n_runs` — počet běhů
* `n_coins` — počet mincí v každém běhu
* `strategies` — seznam testovaných strategií

#### Výstup experimentu

* `df_ret` — tabulka návratnosti běhů pro všechny strategie

#### Průběh experimentu


In [ ]:
# Doplňte kód pro výpočet průměrné návratnosti strategií během stanoveného počtu nezávislých běhů

n_bandits = 5            # počet banditů
n_coins = 100            # počet mincí 
n_runs = 200             # počet běhů

strategies = []

np.random.seed(0)

returns =                       #! inicializace matice běhy x strategie

for i                           #! pro každý běh

    bandits =                   #! nová sada banditů 

    for j                       #! pro každou strategii

        bandits.reset()         # reset vnitřního stavu bandity
        returns[i, j] = bandits.throw(strategies[j], n_coins)

df_ret = pd.DataFrame(returns, columns=[s.name() for s in strategies])

print(df_ret.head())


### Výsledky experimentu

#### Výnos napříč experimenty

* rozdělení výnosů strategií napříč experimenty graficky

In [ ]:
def label(x, color, label, right=False):
    plt.gca().text(1 if right else 0, .2, label, fontweight="bold", color=color, size=18, ha="right" if right else "left", va="center", transform=plt.gca().transAxes)

g=sns.FacetGrid(pd.melt(df_ret, var_name='strategy'), row="strategy", hue="strategy", 
                  aspect=7, height=2, palette=palette[:df_ret.shape[1]], xlim=(0,n_coins));

g.map(sns.kdeplot, "value", shade=True, alpha=1, lw=1.5, bw=0.2).map(plt.axhline, y=0, lw=4).map(label, x="value")
g.set_titles("") #set title to blank
g.set(yticks=[]) #set y ticks to blank
g.despine(bottom=True, left=True); #remove 'spines'

#### Průměrný výnos

* `mean` — průměrný výnos strategie
* `std` — směrodatná odchylka výnosu strategie
* `std_mean` — směrodatná odchylka odhadu střední hodnoty výnosu


In [ ]:
df_agg = pd.melt(df_ret, var_name='strategy').groupby('strategy').aggregate(
    [np.mean, np.std]).droplevel(0, axis='columns').reset_index().sort_values('mean')

df_agg['std_mean'] = df_agg['std'] / np.sqrt(n_runs)

df_agg

#### Očekávaný výnos

* odhad střední hodnoty výnosu pro jednotlivé strategie

In [ ]:
n_std = 3

plt.figure(figsize=(12,4))
sns.barplot(x='strategy', y='mean', data=df_agg, palette=palette[:df_ret.shape[0]]).set(title='Odhad očekávaného výnosu', ylim=(0, n_coins));
plt.errorbar(x=df_agg['strategy'], y=df_agg['mean'], yerr=n_std*df_agg["std_mean"], fmt="none", c= "k", capsize=4);

#### Podíl vítězství

* podíl případů, kdy se strategie ukázala být vítězná


In [ ]:
n_std = 3

# Doplňte kód, který spočte, v kolika procentech případů byla daná strategie nejlepší
#df_wins[strategy, wins, err]

plt.figure(figsize=(12,4))
sns.barplot(x='strategy', y='wins', data=df_wins, palette=palette[:df_wins.shape[0]]).set(title='Počet vítězství');
plt.errorbar(x=df_wins['strategy'], y=df_wins['wins'], yerr=df_wins["err"], fmt="none", c= "k", capsize=4);

### Ztráta na vítěze

In [ ]:
# Doplňte kód, který spočte průměrnou ztrátu na vítěze dané strategie.

df_los=

g=sns.FacetGrid(pd.melt(df_los, var_name='strategy'), row="strategy", hue="strategy", 
                  aspect=7, height=2, palette=palette[:df_ret.shape[1]], xlim=(0,n_coins));

g.map(sns.kdeplot, "value", shade=True, alpha=1, lw=1.5, bw=0.2);


In [ ]:
df_los_agg = pd.melt(df_los, var_name='strategy').groupby('strategy').aggregate(
    [np.mean, np.std]).droplevel(0, axis='columns').reset_index().sort_values('mean')

df_los_agg['std_mean'] = df_los_agg['std'] / np.sqrt(n_runs)

df_los_agg

### Vnitřní reprezentace Bayesovské strategie

* beta rozdělení pro vzorkování po stanoveném počtu kroků


In [ ]:
from bandits import BayesStrategy
import scipy.stats as stats

np.random.seed(0)

n_coins = 1000
n_sample = 10000
bandits = Bandits(n_bandits)

print(bandits)

bayes_strategy = BayesStrategy(n_bandits)

for i in range(n_coins):
    bandits.throw(bayes_strategy, 1)

sampling = np.zeros(n_bandits)    
    
for i in range(n_sample):
    sampling[bayes_strategy(bandits.thrown, bandits.returned)]+=1   

x = np.arange(0,1,0.001)
db = pd.DataFrame({'x':x})
for i in range(n_bandits):
    db[f'bandit_{i}'] = stats.beta.pdf(x, bayes_strategy.alphas[i], bayes_strategy.betas[i])

In [ ]:
db_smpl=pd.melt(db, id_vars='x', var_name='bandit')
db_smpl['smpl']='1'
g=sns.FacetGrid(db_smpl, row="bandit", hue="bandit", aspect=7, height=2)
g.map(sns.lineplot, 'x', "value").map(label, "bandit").map(label, 'bandit', label='smpl', right=True).set_titles("").set(yticks=[]);
g.despine(bottom=False, left=True); 

for ax, pos in zip(g.axes.flat, [b.rate for b in bandits.bandits]):
    ax.axvline(x=pos, linestyle='--', lw=2)

In [ ]:
db_smpl